In [ ]:
import sys
import os
import importlib
import pandas as pd
import numpy as np
import geopandas
import seaborn
#sys.path.insert(0, os.path.join(os.environ["DESEABEL_PATH"], "ocean_ecosystem"))
from ocean_ecosystem import noise_impactor
import simulator
import marine_fauna
import marine_map
import matrix_conversion
importlib.reload(matrix_conversion)
importlib.reload(marine_map)
importlib.reload(marine_fauna)
importlib.reload(simulator)
importlib.reload(noise_impactor)

from noise_impactor import FishingBoat, PileDriving
from simulator import Simulator
from marine_fauna import Fish
from marine_map import MarineMap
from matrix_conversion import *

In [ ]:
from pip.req import parse_requirements
install_reqs = parse_requirements('../requirements.txt', session='hack')

## Create 3 boats and one fish species

In [ ]:
boat1 = FishingBoat(id=1, lat=45.20, lon=-5.2, speed=10, length=10)
boat2 = FishingBoat(id=2, lat=44.85, lon=-4.9, speed=15, length=13)
boat3 = FishingBoat(id=3, lat=44.5, lon=-5.2, speed=17, length=5)

path_data = "s3://deseabel/marine_fauna/fish.geojson"

fish = Fish(path_data)

## Initialize a marine map with specific shapes

In [ ]:
# Example
width = 300000
height = 300000
step = 1000
longitude_west = -6.40953
latitude_north = 46.41813

marine_map = MarineMap(width, height, step, longitude_west, latitude_north)

## Initialize a Simulator

In [ ]:
simu = Simulator(marine_map, list_noise_impactor=[boat1, boat2], list_marine_fauna=[])

# Add manually other objects
simu.add_noise_impactor(boat3)
simu.add_marine_fauna(fish)

#### Plot the heatmap of the noise from the map

In [ ]:
#marine_map.matrix_decibel_gpd.to_file(".data/matrix_decibel.geojson", driver="GeoJSON")
seaborn.heatmap(marine_map.matrix_decibel)

## Compute the marine fauna impact for the fish species

In [ ]:
simu.update_marine_fauna_impact("fish")

#### Show different results
1. Plot the corresponding quantified matrix of the different levels of noise impacts
2. Print the corresponding array impact
3. Plot The corresponding location of the fish species

In [ ]:
#marine_map.matrix_decibel_impact_quantified_gpd.to_file("../data/matrix_decibel_impact_quantified", driver="GeoJSON")
seaborn.heatmap(marine_map.matrix_decibel_impact_quantified)

In [ ]:
# with open(f"../data/array_impact_{fish._species}.json", "w") as f_out:
#     json.dump({i: impact for i, impact in enumerate(fish.array_impact)}, f_out)
fish.array_impact

In [ ]:
coords = marine_map.get_coords_xy_from_geopandas(fish.spot_gpd)
matrix_res = np.zeros(marine_map.matrix_decibel.shape)

matrix_res[coords[:, 0], coords[:, 1]] = 1

seaborn.heatmap(matrix_res)

## Use the API

In [ ]:
import requests
import os
import time
zone = "la_rochelle"
path_data = "s3://deseabel/marine_fauna/fish.geojson"
species = "fish"

# urls
url_base = "http://0.0.0.0:8000/"
url_show = os.path.join(url_base, f"show/{zone}")
url_init = os.path.join(url_base, "initialize_user")
url_add_boat = os.path.join(url_base, f"add_boat/fishing_boat?zone={zone}")
url_add_marine_fauna = os.path.join(url_base, f"add_marine_fauna/fish?zone={zone}&geojson_path={path_data}")
url_update_impact = os.path.join(url_base, f"update_marine_fauna_impact?zone={zone}&species={species}")
url_decibel_matrix = os.path.join(url_base, f"decibel_matrix?zone={zone}")
url_decibel_matrix_impact_quantified = os.path.join(url_base, f"decibel_matrix_impact_quantified?zone={zone}")
url_percentage_marine_fauna_impact_by_level = os.path.join(url_base, f"percentage_marine_fauna_impact_by_level?zone={zone}&species={species}")

requests.get(url_base).text

In [ ]:
token1 = requests.post(url_init).json()["token"]
time.sleep(1)
token2 = requests.post(url_init).json()["token"]
header1 = {"token": token1}
header2 = {"token": token2}

In [ ]:
boat1 = {"id": 1, "lat": 45.20, "lon": -5.2, "speed": 10, "length": 10}
boat2 = {"id": 2, "lat": 44.85, "lon": -4.9, "speed": 15, "length": 13}
boat3 = {"id": 3, "lat": 44.5, "lon": -5.2, "speed": 17, "length": 5}
requests.post(url_add_boat, json=boat1, headers=header1)
requests.post(url_add_boat, json=boat2, headers=header1)
requests.post(url_add_boat, json=boat3, headers=header1)
# id_to_del = 2
# url_delete_boat = os.path.join(url_base, f"remove_noise_impactor?id={id_to_del}")
# requests.get(url_delete_boat).text

In [ ]:
requests.get(url_show, headers=header1).text

In [ ]:
requests.get(url_show, headers=header2).text

In [ ]:
requests.post(url_add_marine_fauna, headers=header1).text
# species_to_del = "fish"
# url_delete_species = os.path.join(url_base, f"remove_marine_fauna?species={species_to_del}")
# requests.delete(url_delete_species).text

In [ ]:
requests.get(url_show, headers=header1).text

In [ ]:
requests.post(url_update_impact, headers=header1)

In [ ]:
res = requests.get(url_decibel_matrix, headers=header1)

In [ ]:
res = requests.get(url_decibel_matrix_impact_quantified, headers=header1)

In [ ]:
res = requests.get(url_percentage_marine_fauna_impact_by_level, headers=header1)

In [ ]:
res.text